# Prototyping

Creates a neural network which evaluates a time series and produce a set of predicted values for the time series
Predicted values may be used in a policy to make a trade. This policy may be modeled by simple multiple regression or a neural network.

## Data
Test data is taken as most recent to avoid lookahead bias. Train data is split into a validation and training set during fitting.


## TODO
- Convert feature percentages to stdev
- Adding VIX as a signal
- Adding High/Low as signals
- Multiple securities/ aggregate samples
- Policy network
- Regularization (l2)
- Dilated convolution

In [365]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers

dataset_path = keras.utils.get_file("IBM.csv", "http://localhost:8000/data/daily/IBM.csv")

column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
raw_dataset = pd.read_csv(dataset_path, 
#                           names=column_names, 
                          dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                          header=0,
                          na_values = "?", 
                          comment='\t',
                          sep=",",
                          skipinitialspace=True)


dataset = raw_dataset.copy()
dataset = dataset.sort_values(by=['Date'],ascending=False)

dataset_stats = dataset.describe()
dataset_stats = dataset_stats.transpose()
NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 3
NUM_SAMPLES = len(dataset)
NUM_TEST_SAMPLES = round(.3 * NUM_SAMPLES,0)
print NUM_SAMPLES
# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old


def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation, 10th of percentage
    return map(lambda arr: map(lambda val: round(val,2),arr), labels)

def convert_to_train(raw_dataset):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(5, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
            dataset['Close'].iloc[i-3] / latest_close, # 3 day trade
            dataset['Close'].iloc[i-5] / latest_close, # 5 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]
converted_feature_set = convert_to_train(dataset)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(converted_feature_set[0][0])]),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(NUM_OUTPUT_NEURONS)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer='sgd',
                metrics=[
                    'mae',
                #    'accuracy'
                ])
  return model

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model = build_model()
model.summary()



868352/856628 [==============================] - 0s 0us/step
12278
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_102 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_103 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_104 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_105 (Dense)            (None, 3)                 51        
Total params: 6,819
Trainable params: 6,819
Non-trainable params: 0
_________________________________________________________________


In [370]:
#len(converted_feature_set[0][0])
print NUM_SAMPLES * .3
NUM_TEST_SAMPLES = 3000
train_data = np.array(converted_feature_set[0][NUM_TEST_SAMPLES:])
train_labels = np.array(converted_feature_set[1][NUM_TEST_SAMPLES:])

test_data = np.array(converted_feature_set[0][:NUM_TEST_SAMPLES])
test_labels = np.array(converted_feature_set[1][:NUM_TEST_SAMPLES])


3683.4


In [371]:
print train_data
print train_labels

[[ 0.         -0.00492553 -0.02721339 ...  0.05639702  0.04999381
   0.05060954]
 [ 0.         -0.02217862 -0.00318591 ...  0.05465015  0.05526287
   0.05416002]
 [ 0.          0.01858062  0.01414529 ...  0.07576121  0.07468228
   0.06137611]
 ...
 [ 0.          0.01234574  0.01080253 ... -0.13888888 -0.14043203
  -0.13811721]
 [ 0.         -0.0015625  -0.02031256 ... -0.1546875  -0.15234375
  -0.15156244]
 [ 0.         -0.01872081 -0.03588144 ... -0.15054602 -0.14976593
  -0.14898596]]
[[1.01 1.01 1.01]
 [1.   1.01 1.  ]
 [0.98 0.98 0.98]
 ...
 [1.01 1.01 1.  ]
 [1.01 1.04 1.01]
 [1.   1.02 1.02]]


In [372]:

history = model.fit(
    train_data, train_labels,
    epochs=60, validation_split = 0.2, verbose=1,
    callbacks=[tensorboard]
)

Train on 7367 samples, validate on 1842 samples
Epoch 1/60
7367/7367 [==============================] - 1s 101us/step - loss: 0.1570 - mean_absolute_error: 0.2734 - val_loss: 0.0039 - val_mean_absolute_error: 0.0485
Epoch 2/60
7367/7367 [==============================] - 0s 58us/step - loss: 0.0035 - mean_absolute_error: 0.0450 - val_loss: 0.0018 - val_mean_absolute_error: 0.0322
Epoch 3/60
7367/7367 [==============================] - 0s 59us/step - loss: 0.0023 - mean_absolute_error: 0.0356 - val_loss: 0.0014 - val_mean_absolute_error: 0.0281
Epoch 4/60
7367/7367 [==============================] - 0s 54us/step - loss: 0.0019 - mean_absolute_error: 0.0325 - val_loss: 0.0013 - val_mean_absolute_error: 0.0267
Epoch 5/60
7367/7367 [==============================] - 0s 61us/step - loss: 0.0017 - mean_absolute_error: 0.0310 - val_loss: 0.0012 - val_mean_absolute_error: 0.0259
Epoch 6/60
7367/7367 [==============================] - 0s 59us/step - loss: 0.0016 - mean_absolute_error: 0.0300 - 

7367/7367 [==============================] - 0s 56us/step - loss: 0.0010 - mean_absolute_error: 0.0231 - val_loss: 7.8961e-04 - val_mean_absolute_error: 0.0206
Epoch 49/60
7367/7367 [==============================] - 0s 56us/step - loss: 0.0010 - mean_absolute_error: 0.0230 - val_loss: 7.8474e-04 - val_mean_absolute_error: 0.0205
Epoch 50/60
7367/7367 [==============================] - 0s 59us/step - loss: 0.0010 - mean_absolute_error: 0.0230 - val_loss: 7.8481e-04 - val_mean_absolute_error: 0.0205
Epoch 51/60
7367/7367 [==============================] - 0s 64us/step - loss: 0.0010 - mean_absolute_error: 0.0230 - val_loss: 7.8191e-04 - val_mean_absolute_error: 0.0205
Epoch 52/60
7367/7367 [==============================] - 0s 60us/step - loss: 0.0010 - mean_absolute_error: 0.0229 - val_loss: 7.8062e-04 - val_mean_absolute_error: 0.0205
Epoch 53/60
7367/7367 [==============================] - 0s 57us/step - loss: 0.0010 - mean_absolute_error: 0.0229 - val_loss: 7.7700e-04 - val_mean_abs

In [373]:
outputs = model.predict(test_data)


In [374]:

# one day return
actual = map(lambda arr: arr[0],outputs)
# signal step for our policy network
signals = map(lambda arr: sum(arr) / len(arr),outputs)
# primitive policy temporarily in place of a RL policy network
trades = map(lambda signal: 1 if round(signal,2) > 1 else 0,signals)

In [375]:
df = pd.DataFrame({
    'signal':signals,
#    'actual':test_labels,
    'actual': actual,
    'trade':trades,
})

In [376]:
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)


In [377]:
# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df

,actual,signal,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,0.990785,0.989426,0,0,0,1,0,1
1,0.996208,0.998264,0,0,0,1,0,1
2,0.989342,0.990591,0,0,0,1,0,1
3,0.982030,0.988606,0,0,0,1,0,1
4,0.989267,0.994969,0,0,0,1,0,1
5,0.998989,1.005613,1,0,1,0,0,0
6,0.998928,1.003428,0,0,0,1,0,1
7,0.996992,0.999238,0,0,0,1,0,1
8,0.992286,0.994585,0,0,0,1,0,1
9,1.002133,1.005099,1,1,0,0,0,1


In [382]:
print '\nNon-loss events'
print sum(df['success'])
print sum(df['success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nLose trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nMissed opportunities'
print sum(df['avoid_failure'])
print sum(df['avoid_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nBullets dodged'
print sum(df['avoid_success'])
print sum(df['avoid_success']) / (NUM_TEST_SAMPLES * 1.00)


Non-loss events
2927
0.975666666667

Lose trades
73
0.0243333333333

Win trades
873
0.291

Missed opportunities
612
0.204

Bullets dodged
1442
0.480666666667
